In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals, unicode_literals

import os

import tensorflow as tf
from tensorflow import keras

tf.__version__

'1.13.1'

In [4]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [5]:
# 간단한 Sequential 모델을 반환합니다
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation=tf.keras.activations.relu, input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation=tf.keras.activations.softmax)
  ])

  model.compile(optimizer=tf.keras.optimizers.Adam(),
                loss=tf.keras.losses.sparse_categorical_crossentropy,
                metrics=['accuracy'])

  return model


# 모델 객체를 만듭니다
model = create_model()
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [6]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 체크포인트 콜백 만들기
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model = create_model()

model.fit(train_images, train_labels,  epochs = 10,
          validation_data = (test_images,test_labels),
          callbacks = [cp_callback])  # 훈련 단계에 콜백을 전달합니다

# 옵티마이저의 상태를 저장하는 것과 관련되어 경고가 발생할 수 있습니다.
# 이 경고는 (그리고 이 노트북의 다른 비슷한 경고는) 이전 사용 방식을 권장하지 않기 위함이며 무시해도 좋습니다.

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 960/1000 [===========================>..] - ETA: 0s - loss: 1.1479 - acc: 0.6687
Epoch 00001: saving model to training_1/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
Instructions for updating:
Use tf.train.CheckpointManager to manage checkpoints rather than manually editing the Checkpoint proto.
1000/1000 [==============================] - 0s 457us/sample - loss: 1.1258 - acc: 0.6740 - val_loss: 0.7165 - val_acc: 0.7760
Epoch 2/10
 928/1000 [==========================>...] - ETA: 0s - loss: 0.4142 - acc: 0.8793
Epoch 00002: saving model to training_1/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1000/1000 [==============================] - 0s 370us/sample - loss: 0.4166 - acc: 0.8770 - val_loss: 0.5357 - val_acc: 0.8440
Epoch 3/10
 992/1000 [============================>.] - ETA: 0s - loss: 0.3003 - acc: 0.9173
Epoch 00003: saving model to training_1/cp.ckpt

Consider using a TensorFlow opt

In [7]:
!ls {checkpoint_dir}

checkpoint
cp.ckpt.data-00000-of-00001
cp.ckpt.index


In [8]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels)
print("훈련되지 않은 모델의 정확도: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 82us/sample - loss: 2.3617 - acc: 0.1340
훈련되지 않은 모델의 정확도: 13.40%


In [9]:
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images, test_labels)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 45us/sample - loss: 0.4173 - acc: 0.8660
복원된 모델의 정확도: 86.60%


In [10]:
# 파일 이름에 에포크 번호를 포함시킵니다(`str.format` 포맷)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # 다섯 번째 에포크마다 가중치를 저장합니다
    period=5)

model = create_model()
model.save_weights(checkpoint_path.format(epoch=0))
model.fit(train_images, train_labels,
          epochs = 50, callbacks = [cp_callback],
          validation_data = (test_images,test_labels),
          verbose=0)


Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00005: saving model to training_2/cp-0005.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00010: saving model to training_2/cp-0010.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00015: saving model to training_2/cp-0015.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00020: saving model to training_2/cp-0020.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00025: saving model to training_2/cp-0025.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00030: saving model to training_2/cp-0030.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00035: saving model to training_2/cp-0035.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00040: saving model to training_2/cp-0040.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00045: saving model to training_2/cp-0045.ckpt

Consider 

In [11]:
! ls {checkpoint_dir}

checkpoint
cp-0000.ckpt.data-00000-of-00001
cp-0000.ckpt.index
cp-0005.ckpt.data-00000-of-00001
cp-0005.ckpt.index
cp-0010.ckpt.data-00000-of-00001
cp-0010.ckpt.index
cp-0015.ckpt.data-00000-of-00001
cp-0015.ckpt.index
cp-0020.ckpt.data-00000-of-00001
cp-0020.ckpt.index
cp-0025.ckpt.data-00000-of-00001
cp-0025.ckpt.index
cp-0030.ckpt.data-00000-of-00001
cp-0030.ckpt.index
cp-0035.ckpt.data-00000-of-00001
cp-0035.ckpt.index
cp-0040.ckpt.data-00000-of-00001
cp-0040.ckpt.index
cp-0045.ckpt.data-00000-of-00001
cp-0045.ckpt.index
cp-0050.ckpt.data-00000-of-00001
cp-0050.ckpt.index


In [12]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2\\cp-0050.ckpt'

In [13]:
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images, test_labels)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 83us/sample - loss: 0.4994 - acc: 0.8720
복원된 모델의 정확도: 87.20%


In [14]:
# 가중치를 저장합니다
model.save_weights('./checkpoints/my_checkpoint')

# 가중치를 복원합니다
model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss,acc = model.evaluate(test_images, test_labels)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))


Consider using a TensorFlow optimizer from `tf.train`.
1000/1000 [==============================] - 0s 88us/sample - loss: 0.4994 - acc: 0.8720
복원된 모델의 정확도: 87.20%


In [15]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

# 전체 모델을 HDF5 파일로 저장합니다
model.save('my_model.h5')

Epoch 1/5
1000/1000 [==============================] - 0s 337us/sample - loss: 1.1664 - acc: 0.6690
Epoch 2/5
1000/1000 [==============================] - 0s 251us/sample - loss: 0.4369 - acc: 0.8780
Epoch 3/5
1000/1000 [==============================] - 0s 247us/sample - loss: 0.2763 - acc: 0.9260
Epoch 4/5
1000/1000 [==============================] - 0s 245us/sample - loss: 0.1971 - acc: 0.9560
Epoch 5/5
1000/1000 [==============================] - 0s 260us/sample - loss: 0.1557 - acc: 0.9630


In [16]:
# 가중치와 옵티마이저를 포함하여 정확히 동일한 모델을 다시 생성합니다
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [17]:
loss, acc = new_model.evaluate(test_images, test_labels)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 103us/sample - loss: 0.4081 - acc: 0.8680
복원된 모델의 정확도: 86.80%


In [18]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

Epoch 1/5
1000/1000 [==============================] - 0s 350us/sample - loss: 1.2280 - acc: 0.6350
Epoch 2/5
1000/1000 [==============================] - 0s 240us/sample - loss: 0.4533 - acc: 0.8670
Epoch 3/5
1000/1000 [==============================] - 0s 238us/sample - loss: 0.3212 - acc: 0.9080
Epoch 4/5
1000/1000 [==============================] - 0s 247us/sample - loss: 0.2033 - acc: 0.9560
Epoch 5/5
1000/1000 [==============================] - 0s 251us/sample - loss: 0.1538 - acc: 0.9670


In [22]:
saved_model_path = tf.contrib.saved_model.save_keras_model(model, "c:/Projects/Keras_talk")


Consider using a TensorFlow optimizer from `tf.train`.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['serving_default']
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: c:/Projects/Keras_talk\1564128045\saved_model.pb


In [23]:
!ls c:/Projects/Keras_talk

1564128045
MNIST_data
Mnist.ipynb
Projects
Untitled.ipynb
Untitled1.ipynb
Untitled2.ipynb
Untitled3.ipynb
Untitled4.ipynb
Untitled5.ipynb
Untitled6.ipynb
Untitled7.ipynb
Untitled8.ipynb
Untitled9.ipynb
ch6.ipynb
checkpoints
face_recognition-master
keras
keras1.ipynb
lab11-1.ipynb
lab11-2.ipynb
lab11-3.ipynb
lab11-4 layer .ipynb
logs
mnist_mlp_model.h5
my_model.h5
save.ipynb
ship.ipynb
tmp.npy
training_1
training_2


In [24]:
new_model = tf.contrib.saved_model.load_keras_model(saved_model_path)
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [25]:
# 이 모델을 평가하려면 그전에 컴파일해야 합니다.
# 단지 저장된 모델의 배포라면 이 단계가 필요하지 않습니다.

new_model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=['accuracy'])

# 복원된 모델을 평가합니다
loss, acc = new_model.evaluate(test_images, test_labels)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 103us/sample - loss: 0.4230 - acc: 0.8600
복원된 모델의 정확도: 86.00%
